# Notebook to process and clean csv's

In [2]:
import pandas as pd
import numpy as np
import altair as alt

# Performance Predictions for mnist, cifar10 and imagenet - Heatmaps
## Read csv > process data > save to another csv

In [5]:
#hide
## Reading csv file and converting data to (Neural network, Platform, Value)
df = pd.read_csv('data/performance_predictions_imagenet_mnist_cifar.csv')
df.head()
pd.set_option('display.max_rows', None)
#----- Creating a dataframe with 3 columns x, y gop_frame
cleanedList = [x for x in df.platform if x==x] # to take all the nans out
x, y = np.meshgrid(df.model, cleanedList) 
gop_frame, _ = np.meshgrid(df.gop_frame, cleanedList)

#to crate a 1D array from each variable, creating a dataframe with 3 columns
source = pd.DataFrame({'x': x.ravel(),     
                       'y': y.ravel(),
                       'gop_frame':gop_frame.ravel()}) #auxilary column

#---Adding a fourth column: top_second  ---- auxilary column
tops_second= []    #creating a lsit which will contain all top_second columns from the dataframe
columns = list(df) # creating a list of dataframe columns 
  
for i in columns:   
    if 'top_second' in i:
        tops_second.append(df[i])
        
source['top_second'] = pd.concat(tops_second,ignore_index=True)

#------Adding a fith column: values-----------
source['values'] = source.top_second * 1000 / source.gop_frame

#---Drop auxilary columns: gop_frame top_scond----
source = source.drop(columns=['gop_frame','top_second'])
source = source.round(0)
source.head()

#Separate dataframe into: IMAGENET, MNIST, CIFAR10 dataframes
df_imagenet = source[source['x'].str.contains('GoogleNetv|MobileNetv1|ResNet50|EfficientNet')]
df_cifar10 = source[source['x'].str.contains('CNV')]
df_MNIST = source[source['x'].str.contains('MLP')]

#Saving above dataframes to csv file
df_imagenet.to_csv('data/processed_csv/performance_prediction_imagenet.csv', index = False)
df_cifar10.to_csv('data/processed_csv/performance_prediction_cifar10.csv', index = False)
df_MNIST.to_csv('data/processed_csv/performance_prediction_mnist.csv', index = False)
source.to_csv('data/processed_csv/performance_prediction_imagenet_mnist_cifar10.csv', index = False)